<a href="https://colab.research.google.com/github/wylhtydtm/Nematode-project/blob/master/xgboost_optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn import preprocessing
import time
import copy
import random
from collections import Counter, defaultdict
from sklearn.model_selection._split import _BaseKFold
from sklearn.preprocessing import LabelEncoder
import pdb

In [ ]:
train_data = pd.read_csv('/content/drive/My Drive/xgboost_dataset_balanced_2/train_X_data_forxgboost.csv',index_col=None)
test_data= pd.read_csv('/content/drive/My Drive/xgboost_dataset_balanced_2/test_X_data_forxgboost.csv',index_col=None)
train_label= pd.read_csv('/content/drive/My Drive/xgboost_dataset_balanced_2/train_y__labelforxgboost.csv',index_col=None)
test_label= pd.read_csv('/content/drive/My Drive/xgboost_dataset_balanced_2/test_y__labelforxgboost.csv',index_col=None)

In [ ]:
train_groups= train_label['drug_type']
train_y=train_label['MOA_group']
train_X= train_data

In [ ]:
train_data.shape 

(2449, 251)

In [ ]:
class StratifiedGroupKFold(_BaseKFold):
    """
    Makes stratified folds according to the class labels y, but each group
    defined in groups, is assigned in one fold (all the points of the group
                                                are assigned to the same fold)
    """
    def __init__(self, n_splits=5, random_seed=None):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.seed = random_seed

    def split(self, X, y, groups, seed=None):
        if seed is None:
            seed = self.seed
        group_counts = pd.DataFrame(
            {'y':y, 'groups':groups}).groupby(by='y').agg({"groups": "nunique"})
        if np.any(group_counts.values<self.n_splits):
            raise ValueError('Some of the classes have less groups than '+
                             'the number of splits.')

        y = LabelEncoder().fit_transform(y)
        labels_num = np.max(y) + 1
        y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
        y_distr = Counter()
        for label, g in zip(y, groups):
            y_counts_per_group[g][label] += 1
            y_distr[label] += 1

        y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
        groups_per_fold = defaultdict(set)

        def eval_y_counts_per_fold(y_counts, fold):
            y_counts_per_fold[fold] += y_counts
            std_per_label = []
            for label in range(labels_num):
                label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(self.n_splits)])
                std_per_label.append(label_std)
            y_counts_per_fold[fold] -= y_counts
            return np.mean(std_per_label)

        groups_and_y_counts = list(y_counts_per_group.items())
        random.Random(seed).shuffle(groups_and_y_counts)

        for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
            best_fold = None
            min_eval = None
            for i in range(self.n_splits):
                fold_eval = eval_y_counts_per_fold(y_counts, i)
                if min_eval is None or fold_eval < min_eval:
                    min_eval = fold_eval
                    best_fold = i
            y_counts_per_fold[best_fold] += y_counts
            groups_per_fold[best_fold].add(g)

        all_groups = set(groups)
        for i in range(self.n_splits):
            train_groups = all_groups - groups_per_fold[i]
            test_groups = groups_per_fold[i]

            train_indices = [i for i, g in enumerate(groups) if g in train_groups]
            test_indices = [i for i, g in enumerate(groups) if g in test_groups]

            yield train_indices, test_indices

In [ ]:
train_groups= train_label['drug_type']
train_y=train_label['MOA_group']
train_X= train_data
k=4

test_groups= test_label['drug_type']
test_y= test_label['MOA_group']
test_X=test_data

In [ ]:
print(train_X.shape)
print(test_X.shape)

(2449, 251)
(723, 251)


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from collections import Counter

In [ ]:
def score_majority_vote(
        y_real, group,
        y_pred=None, probas=None, labels=None,
        vote_type='counts',
        scorer=None
        ):

    if scorer is None:
        score_func = accuracy_score
    else:
        score_func = scorer

    y_real = np.asarray(y_real)
    y_pred = np.asarray(y_pred)
    group = np.asarray(group)

    ugroups = np.unique(group)

    assert np.all([np.unique(y_real[group==g]).shape[0]==1 for g in ugroups]), \
        'The real class labels are not unique per group.'

    y_group = [np.unique(y_real[group==g])[0] for g in ugroups]
    y_maj = get_majority_vote(
        group, y_pred=y_pred, probas=probas, labels=labels, vote_type=vote_type)
    y_maj = y_maj[ugroups]

    return score_func(y_group, y_maj)

In [ ]:
def get_majority_vote(
        group, y_pred=None, probas=None, labels=None, vote_type='counts'):
    """
    Get the majority vote predictions per group.
    param:
        groups: an array defining the groups of data points (array size n_samples)
        y_pred: the predicted class labels for each data point (array size n_samples)
        probas: the probabilities for each class for each data point (array shape=(n_samples, n_classes) )
        labels: the class labels that match each column of the probas array
        vote_type: the rule the the majoroty vote is based on ['counts', 'probas']
    """

    if probas is not None and labels is None:
        raise ValueError('Must provide class labels corresponding to the '+
                         'columns of the probas.')

    if vote_type == 'counts' and y_pred is None:
        if probas is None:
            raise ValueError('Must provide either y_pred or probas to use the counts sum rule.')
        else:
            y_pred = [labels[maxind] for maxind in np.argmax(probas, axis=1)]
            y_pred = np.array(y_pred)

    if vote_type == 'counts' and labels is not None:
        assert all([pred in labels for pred in y_pred]), \
            'The predictions in y_pred do not match the labels provided.'

    if vote_type == 'counts':
        y_maj = {}
        for grp in np.unique(group):
            c = Counter(y_pred[group==grp])

            #value,count = c.most_common()[0]
            counts=np.array([votes for clss,votes in c.most_common()])

            if (sum(counts==counts[0])==1) or (probas is None):
                value,count = c.most_common()[0]
                y_maj[grp] = value
            else:   # if more than one labels have the same number of votes and we have proba info
                assert len(labels)==probas.shape[1]
                values = np.array([clss for clss,votes in c.most_common()])
                equal_classes = values[counts==counts[0]]
                probas_of_equal_classes = []
                for iclass in equal_classes:
                    probas_of_equal_classes.append(
                        np.mean(probas[group==grp, labels==iclass]))
                most_likely_class = equal_classes[np.argmax(probas_of_equal_classes)]
                y_maj[grp] = most_likely_class
        y_maj = pd.Series(y_maj)
    elif vote_type == 'probas':
        labels = np.array(labels).reshape(-1)
        assert labels.shape[0]==probas.shape[1]
        group_probas = pd.DataFrame(probas).groupby(by=group).sum()
        y_maj = pd.Series({grp:labels[np.argmax(group_probas.loc[grp,:].values)]
                           for grp in np.unique(group)})

    return y_maj

In [ ]:
splitter=StratifiedGroupKFold(n_splits=k,random_seed=111)
from sklearn.model_selection import ParameterGrid
accuracy_list=[]
score_func_list=[]

param_test0 = {
 'n_estimators':[300,500,600,700,800,1000,1200]
}

param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

param_test2 = {
 'max_depth':[3,4,5,6,7,8],
 'min_child_weight':[2,3,4,5,6,7]
}

for para in ParameterGrid(param_test4):
  acc_para=[]
  acc_score_func=[]
  for train_ind, test_ind in splitter.split(train_X,train_y,train_groups):
    X_train= train_X.iloc[train_ind,:]
    y_train= train_y[train_ind]
    X_val= train_X.iloc[test_ind,:]
    y_val=train_y[test_ind]
    groups_val= train_groups[test_ind]
    model= XGBClassifier(n_estimators=800,max_depth= 3, min_child_weight=2,colsample_bytree= 0.6, subsample= 0.6,
                         reg_alpha= 1,objective= 'multi:softmax',num_class=12,nthread=4, scale_pos_weight=1, seed=27,**para)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    predictions= [round(value) for value in y_pred]
    accuracy = accuracy_score(y_val, predictions)
    acc_para.append(accuracy)
    score_func=score_majority_vote(y_val,groups_val,y_pred=y_pred)
    acc_score_func.append(score_func)

  print("Accuracy: %.2f%%" %(np.mean(acc_para)*100))
  print("Majority vote accuracy:.%.2f%%" %(np.mean(acc_score_func)*100))

  accuracy_list.append(np.mean(acc_para))
  score_func_list.append(np.mean(acc_score_func))

In [ ]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}

param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}

param_test5 = {
 'reg_alpha':[ 1e-2, 0.1, 1,1.2,1.5,1.8, 2]
}

param_test6 = {
 'learning_rate':[1e-3, 1e-2, 5e-2,0.1, 0.2,0.3]
}

In [ ]:
max_ind=np.argmax(score_func_list)
parameters=list(ParameterGrid(param_test4))[max_ind]
print(parameters)
print(max_ind)
print(list(ParameterGrid(param_test3)))
print(score_func_list)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

In [ ]:
splitter=StratifiedGroupKFold(n_splits=k,random_seed=113)

accuracy_list=[]
score_func_list=[]

for train_ind, test_ind in splitter.split(train_X,train_y,train_groups):
  X_train= train_X.iloc[train_ind,:]
  y_train= train_y[train_ind]
  X_val= train_X.iloc[test_ind,:]
  y_val=train_y[test_ind]
  groups_val= train_groups[test_ind]
  model= XGBClassifier(n_estimators=800,max_depth= 3, min_child_weight=2,colsample_bytree= 0.6, subsample= 0.6,
                       reg_alpha= 1,objective= 'multi:softmax',num_class=12,nthread=4, scale_pos_weight=1, seed=27)

  model.fit(X_train, y_train)
  y_pred = model.predict(X_val)
  predictions= [round(value) for value in y_pred]
  accuracy = accuracy_score(y_val, predictions)
  score_func=score_majority_vote(y_val,groups_val,y_pred=y_pred)
  accuracy_list.append(accuracy)
  score_func_list.append(score_func)
    
print("Accuracy: %.2f%%" %(np.mean(accuracy_list)*100))
print("Majority vote accuracy:.%.2f%%" %(np.mean(score_func_list)*100))

In [ ]:
!pip install xgboost
model= XGBClassifier(learning_rate=0.1,n_estimators=300,max_depth= 7, min_child_weight= 3,gamma=0,reg_alpha=1,
                       colsample_bytree= 0.6, subsample= 0.8,objective= 'multi:softmax',num_class=12,nthread=4, scale_pos_weight=1, seed=27

model= XGBClassifier(learning_rate=0.1,max_depth=3, min_child_weight= 5,n_estimators=1000,gamma=0,reg_alpha=1,
                     colsample_bytree= 0.6, subsample= 0.8,objective= 'multi:softmax',num_class=12,nthread=4, scale_pos_weight=1, seed=27)

In [ ]:
from xgboost import plot_importance
plt.rcParams['figure.figsize'] = [90, 50]
plot_importance(model)
plt.show()

In [ ]:
def get_majority_vote(
        group, y_pred=None, probas=None, labels=None, vote_type='counts'):
    """
    Get the majority vote predictions per group.
    param:
        groups: an array defining the groups of data points (array size n_samples)
        y_pred: the predicted class labels for each data point (array size n_samples)
        probas: the probabilities for each class for each data point (array shape=(n_samples, n_classes) )
        labels: the class labels that match each column of the probas array
        vote_type: the rule the the majoroty vote is based on ['counts', 'probas']
    """

    if probas is not None and labels is None:
        raise ValueError('Must provide class labels corresponding to the '+
                         'columns of the probas.')

    if vote_type == 'counts' and y_pred is None:
        if probas is None:
            raise ValueError('Must provide either y_pred or probas to use the counts sum rule.')
        else:
            y_pred = [labels[maxind] for maxind in np.argmax(probas, axis=1)]
            y_pred = np.array(y_pred)

    if vote_type == 'counts' and labels is not None:
        assert all([pred in labels for pred in y_pred]), \
            'The predictions in y_pred do not match the labels provided.'

    if vote_type == 'counts':
        y_maj = {}
        for grp in np.unique(group):
            c = Counter(y_pred[group==grp])

            #value,count = c.most_common()[0]
            counts=np.array([votes for clss,votes in c.most_common()])

            if (sum(counts==counts[0])==1) or (probas is None):
                value,count = c.most_common()[0]
                y_maj[grp] = value
            else:   # if more than one labels have the same number of votes and we have proba info
                assert len(labels)==probas.shape[1]
                values = np.array([clss for clss,votes in c.most_common()])
                equal_classes = values[counts==counts[0]]
                probas_of_equal_classes = []
                for iclass in equal_classes:
                    probas_of_equal_classes.append(
                        np.mean(probas[group==grp, labels==iclass]))
                most_likely_class = equal_classes[np.argmax(probas_of_equal_classes)]
                y_maj[grp] = most_likely_class
        y_maj = pd.Series(y_maj)
    elif vote_type == 'probas':
        labels = np.array(labels).reshape(-1)
        assert labels.shape[0]==probas.shape[1]
        group_probas = pd.DataFrame(probas).groupby(by=group).sum()
        y_maj = pd.Series({grp:labels[np.argmax(group_probas.loc[grp,:].values)]
                           for grp in np.unique(group)})

    return y_maj

In [ ]:
def get_two_most_likely_majority_vote(y_pred,groups):
    """
    Estimate the classification accuracy when groups of data points are classified together based on a majority vote.
    param:
        y_true: the true class labels of the samples (array size n_samples)
        y_pred: the predicted class labels from the classfier (array size n_samples)
        groups: an array defining the groups of data points (array size n_samples)
    """

    y_maj = np.empty((y_pred.shape[0],2))

    for grp in np.unique(groups):

        c = Counter(y_pred[groups==grp])

        if len(c.most_common())>1:
            for rnk in [0,1]:
                value,count = c.most_common()[rnk]
                y_maj[groups==grp,rnk] = value
        else:
            value,count = c.most_common()[0]
            y_maj[groups==grp,:] = [value,value]

        # if more than one labels have the same number of votes
        if len(c.most_common())>1 and c.most_common()[0][1]==c.most_common()[1][1]:
            print('Warning: the samples of compound {} are classified in more than one classes with the same frequency.'.format(grp))

    return y_maj

In [ ]:
def get_two_most_likely_accuracy(ytest, ytest_pred_two):

    check=[]
    for i,y in enumerate(ytest):
        if y in ytest_pred_two[i]:
            check.append(True)
    acc = np.sum(check)/len(check)
    return acc

In [ ]:
def _get_y_group(y, group):

    y_group = pd.DataFrame(y).groupby(by=group).apply(lambda x: np.unique(x))

    assert all([len(x)==1 for x in y_group.values]), 'y is not unique in each group'

    y_group = y_group.apply(lambda x:x[0])

    return y_group

In [ ]:
def plot_confusion_matrix(
        y_true, y_pred, classes=None, normalize=False, title=None, figsize=(8,8),
        cmap=None, saveto=None
        ):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import matplotlib.pyplot as plt
    from sklearn.metrics import confusion_matrix
    from sklearn.utils.multiclass import unique_labels
    from mpl_toolkits.axes_grid1 import make_axes_locatable

#    if not title:
#        if normalize:
#            title = 'Normalized confusion matrix'
#        else:
#            title = 'Confusion matrix, without normalization'

    if cmap is None:
        cmap = plt.cm.Blues

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    if classes is not None:
        classes = [classes[key] for key in unique_labels(y_true, y_pred)]
    else:
        classes = unique_labels(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    fig, ax = plt.subplots(figsize=figsize)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)

    # create an axes on the right side of ax. The width of cax will be 5%
    # of ax and the padding between cax and ax will be fixed at 0.05 inch.
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    ax.figure.colorbar(im, cax=cax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()

    if saveto is not None:
        plt.savefig(saveto)
        plt.close()
    return


In [ ]:
train_groups= train_label['drug_type']
train_y=train_label['MOA_group']
train_X= train_data

test_groups= test_label['drug_type']
test_y= test_label['MOA_group']
test_X=test_data

In [ ]:
import xgboost as xgb

In [ ]:
#use Dmatrix for xgboost
dtrain=xgb.DMatrix(train_X, label=train_y)
dtest=xgb.DMatrix(test_X, label=test_y)

In [ ]:
param = {'max_depth':3,'eta':0.1,'objective':'multi:softmax','num_class':12,'reg_alpha':1, 'colsample_bytree':0.6, 'subsample': 0.8,'min_child_weight': 5,'max_depth' :3}
epochs=10

In [ ]:
epochs=1000
bst=xgb.train(param, dtrain,epochs)

In [ ]:
preds= bst.predict(dtest)
print(preds)

In [ ]:
accuracy = accuracy_score(test_y, preds)
print("Accuracy: %.2f%%" %(accuracy *100))

Accuracy: 32.50%


In [ ]:
model= XGBClassifier(n_estimators=800,max_depth= 3, min_child_weight=2,colsample_bytree= 0.6, subsample= 0.6,
                       reg_alpha= 1,objective= 'multi:softmax',num_class=12,nthread=4, scale_pos_weight=1, seed=27)
model.fit(train_X, train_y)
y_pred = model.predict(test_X)
predictions= [round(value) for value in y_pred]
accuracy = accuracy_score(test_y, predictions)
print("Accuracy: %.2f%%" %(accuracy *100))

Accuracy: 33.47%


In [ ]:
score_function= score_majority_vote(y_real=test_y, group=test_groups,y_pred=preds)

print("score_function_accuracy: %.2f%%" %(score_function *100))

score_function_accuracy: 55.56%


In [ ]:
vote=get_two_most_likely_majority_vote(y_pred=preds,groups=test_groups)

In [ ]:
acc=get_two_most_likely_accuracy(ytest=test_y, ytest_pred_two=vote)
print("Accuracy: %.2f%%" %(acc *100))

Accuracy: 100.00%


In [ ]:
y_maj = get_majority_vote(test_groups, y_pred=preds)

In [ ]:
y_true= _get_y_group(test_y, test_groups)

In [ ]:
y_maj = y_maj[y_true.index]
print(y_true)

drug_type
CSAA011840     3.0
CSAA016712     3.0
CSAA020364    11.0
CSAA026102     2.0
CSAA118599     9.0
CSAA123408     0.0
CSAA128717     4.0
CSAA375588    11.0
CSAA398011     6.0
CSCC170103    10.0
CSCC201954     7.0
CSCC222657     8.0
CSCC811993     6.0
CSCC812553    10.0
CSCD068947     8.0
CSCD625957     1.0
CSCD668797     5.0
CSCD677230     5.0
dtype: float64


In [ ]:
test_acc = accuracy_score(y_true.values, y_maj.values)
print(test_acc)

0.5555555555555556


In [ ]:
plot_confusion_matrix(y_true.values, y_maj.values,
                      title='Test accuracy = %.2f%%'% (test_acc*100))